In [122]:
import pandas as pd
from datetime import datetime

In [123]:
def payer_category(payer_category):
    cats = {
        'DISCOUNTED CASH PRICE INPATIENT': 'cash', 
        'DISCOUNTED CASH PRICE INPATIENT': 'cash', 
        'Gross Charge': 'gross',
        'DE-IDENTIFIED MIN': 'min',
        'DE-IDENTIFIED MAX': 'max'
    }
    try:
        return cats[payer_category]
    except KeyError:
        return "payer"

In [124]:
file = "621811413_South Baldwin_standardcharges.csv"
ccn = '010083'
url = 'https://www.southbaldwinrmc.com/Uploads/Public/Documents/charge-masters/621811413_South%20Baldwin_standardcharges.csv'

In [125]:
df = pd.read_csv(file)

In [126]:
df = df.rename(columns={
    "SVCCD": 'procedure_code',
    'Description': 'description',
    'CPT': 'code',
    'As of Date': 'file_last_updated'
})

In [127]:
df['hcpcs_cpt'] = df['code'].apply(lambda x: x if len(str(x)) == 5 else 'na')
df["file_last_updated"] = pd.to_datetime(df["file_last_updated"])

In [128]:
df

,procedure_code,description,code,Gross Charge,file_last_updated,DISCOUNTED CASH PRICE INPATIENT,DISCOUNTED CASH PRICE OUTPATIENT,DE-IDENTIFIED MIN,DE-IDENTIFIED MAX,BCBSAL PPO - All Plans,...,Humana PPO - All Plans,COMM Viva Health PPO - All Plans,UHC MA - All Plans,Humana MA - All Plans,BCBSAL MA - All Plans,Healthspring MA - All Plans,Aetna/Coventry MA - All Plans,Viva Health MA - All Plans,Multiplan - All Plans,hcpcs_cpt
0,8650033,TC99M PENTETAT-DOSE,A9539,5352.840000,2023-02-13,1605.852000,963.511200,369.345960,4068.158400,406.815840,...,1707.555960,2408.778000,390.757320,369.345960,396.110160,396.110160,428.227200,412.168680,1391.738400,A9539
1,1314655,BORRELIA BURG.CONFIR,86617,773.110000,2023-02-13,231.933000,139.159800,53.344590,587.563600,58.756360,...,246.622090,347.899500,56.437030,53.344590,57.210140,57.210140,61.848800,59.529470,201.008600,86617
2,1973235,"THROMBOLY,CORONRY,IV",92977,2596.340000,2023-02-13,778.902000,467.341200,179.147460,1973.218400,197.321840,...,828.232460,1168.353000,189.532820,179.147460,192.129160,192.129160,207.707200,199.918180,675.048400,92977
3,1621628,US-PLC LOC DVCE 1ST,MANUL,1784.530000,2023-02-13,535.359000,321.215400,123.132570,1356.242800,135.624280,...,569.265070,803.038500,130.270690,123.132570,132.055220,132.055220,142.762400,137.408810,463.977800,MANUL
4,1211040,GENTAMICIN UPTO 80MG,J1580,17.940000,2023-02-13,5.382000,3.229200,1.237860,13.634400,1.363440,...,5.722860,8.073000,1.309620,1.237860,1.327560,1.327560,1.435200,1.381380,4.664400,J1580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5433,SURG,Ca screen;flexi sigmoidscope,G0104,11908.875000,2023-02-13,3572.662500,2143.597500,821.712375,9050.745000,905.074500,...,3798.931125,5358.993750,869.347875,821.712375,881.256750,881.256750,952.710000,916.983375,3096.307500,G0104
5434,SURG,Colorectal scrn; hi risk ind,G0105,12905.830870,2023-02-13,3871.749261,2323.049557,890.502330,9808.431461,980.843146,...,4116.960047,5807.623891,942.125653,890.502330,955.031484,955.031484,1032.466470,993.748977,3355.516026,G0105
5435,SURG,Colon ca scrn not hi rsk ind,G0121,12080.911730,2023-02-13,3624.273520,2174.564112,833.582910,9181.492917,918.149292,...,3853.810843,5436.410280,881.906556,833.582910,893.987468,893.987468,966.472939,930.230204,3141.037051,G0121
5436,SURG,Inj for sacroiliac jt anesth,G0260,8695.808846,2023-02-13,2608.742654,1565.245592,600.010810,6608.814723,660.881472,...,2773.963022,3913.113981,634.794046,600.010810,643.489855,643.489855,695.664708,669.577281,2260.910300,G0260


In [129]:
cols = df.columns.tolist()
id_vars = cols[:cols.index('hcpcs_cpt')+1]
id_vars.extend(['file_last_updated', 'code'])

payer_cat = cols[cols.index('DISCOUNTED CASH PRICE INPATIENT'):]
payer_cat.pop(-1) # Remove code from list
payer_cat.append('Gross Charge')

df = pd.melt(df, id_vars=id_vars, value_vars=payer_cat, var_name='payer_desc', value_name='rate')

ValueError: negative dimensions are not allowed

In [ ]:
df["payer_category"] = df["payer_desc"].apply(payer_category)

In [ ]:
df['patient_class'] = df['payer_desc'].apply(lambda x: 'inpatient' if x.endswith('INPATIENT') else ('outpatient' if x.endswith('OUTPATIENT') else 'na'))

In [ ]:
df["plan_name"] = df['payer_desc'].apply(lambda x: x.split(" - ")[-1].strip() if " - " in x else pd.NA)

df["code_type"] = df['code'].apply(lambda x: 'hcpcs_cpt' if len(str(x)) == 5 else 'na')

In [ ]:
tin = file.split("_")[0]
tin = tin[:2] + "-" + tin[2:]

df["filename"] = file
df["url"] = url
df["hospital_ccn"] = ccn
df["hospital_tin"] = tin

In [ ]:
df["payer_desc"] = df["payer_desc"].str.strip()

In [ ]:
print(len(df))
df.dropna(subset=['code'], inplace=True)
print(len(df))

In [ ]:
df.to_csv("south_baldwin.csv", index=False)